In [1]:
import pandas as pd
import numpy as np
import json
from yelpapi import YelpAPI
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# Load Yelp API token data
f = open('access_tokens.json', 'r')
tokens = json.load(f)
section_tokens = tokens['yelp']['first_yelp_app_rwilleyii']
client_id = section_tokens['Client ID']
api_key = section_tokens['API Key']

# Load Yelp API
yelp_api = YelpAPI(api_key)

In [2]:
# Get user preferences
company_name = input('Please enter company name as it appears on Yelp.com: ')
term = input('For someone who has never been to your restaurant, what terms might they search on Yelp to find you? ')
location = input('Where is your restaurant located? (city ST, ZIP, etc.) ')

Please enter company name as it appears on Yelp.com:  Chano's Cantina
For someone who has never been to your restaurant, what terms might they search on Yelp to find you?  mexican food
Where is your restaurant located? (city ST, ZIP, etc.)  astoria ny


In [3]:
search_limit = 50

url = 'https://api.yelp.com/v3/businesses/search'

headers = {'Authorization': 'Bearer {}'.format(api_key)}

url_params = {'term': term.replace(' ', '+'),
              'location': location.replace(' ', '+'),
              'limit': search_limit
             }
response = yelp_api.search_query(term = term,
                                 location = location,
                                 limit = search_limit)

In [4]:
cols = list(response['businesses'][0].keys())
data = pd.DataFrame(columns=cols)

for biz in response['businesses']:
    data = data.append(biz, ignore_index=True)

data = data[data['is_closed']==False]
data.head(10)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,6AJwsgXr7YwsqneGVAdgzw,las-catrinas-mexican-bar-and-eatery-astoria,Las Catrinas Mexican Bar & Eatery,https://s3-media3.fl.yelpcdn.com/bphoto/3uevye...,False,https://www.yelp.com/biz/las-catrinas-mexican-...,303,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 40.7614214682633, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '32-02 Broadway', 'address2': '',...",+19177450969,(917) 745-0969,642.525771
1,jzVv_21473lAMYXIhVbuTA,de-mole-astoria-astoria,De Mole Astoria,https://s3-media1.fl.yelpcdn.com/bphoto/zZTTfd...,False,https://www.yelp.com/biz/de-mole-astoria-astor...,359,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,"{'latitude': 40.7625999, 'longitude': -73.9129...","[delivery, pickup]",$$,"{'address1': '4220 30th Ave', 'address2': '', ...",+17187771655,(718) 777-1655,918.092772
2,2jjxdN9FXWVdx6DEX83krQ,hoja-santa-astoria,Hoja Santa,https://s3-media3.fl.yelpcdn.com/bphoto/xqo13E...,False,https://www.yelp.com/biz/hoja-santa-astoria?ad...,32,"[{'alias': 'newmexican', 'title': 'New Mexican...",4.5,"{'latitude': 40.7638197, 'longitude': -73.914523}",[],NaN,"{'address1': '40-17 30th Ave', 'address2': '',...",,,751.531469
3,AUyKmFjpaVLwc3awfUnqgQ,chela-and-garnacha-astoria,Chela & Garnacha,https://s3-media1.fl.yelpcdn.com/bphoto/ChVbA1...,False,https://www.yelp.com/biz/chela-and-garnacha-as...,363,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.5,"{'latitude': 40.7557171543477, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '33-09 36th Ave', 'address2': '',...",+19178326876,(917) 832-6876,1318.326547
4,zNcDCPiUsE5aT6jyaCVtmA,frescos-cantina-long-island-city,Fresco's Cantina,https://s3-media2.fl.yelpcdn.com/bphoto/3IdlTT...,False,https://www.yelp.com/biz/frescos-cantina-long-...,207,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.5,"{'latitude': 40.76837, 'longitude': -73.93303}","[delivery, restaurant_reservation, pickup]",$$,"{'address1': '12-14 31st Ave', 'address2': Non...",+13478087338,(347) 808-7338,927.218713
5,QIsFsiOP3H_NkgeWST7GPA,maizal-restaurant-and-tequila-bar-astoria-2,Maizal Restaurant & Tequila Bar,https://s3-media3.fl.yelpcdn.com/bphoto/B2n1tz...,False,https://www.yelp.com/biz/maizal-restaurant-and...,294,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 40.759331, 'longitude': -73.926035}","[delivery, pickup]",$$,"{'address1': '3207 34th Ave', 'address2': None...",+17184069431,(718) 406-9431,900.451091
6,yvva7IYpD6J7OfKlCdQrkw,mi-espiguita-taqueria-astoria,Mi Espiguita Taqueria,https://s3-media2.fl.yelpcdn.com/bphoto/TEho39...,False,https://www.yelp.com/biz/mi-espiguita-taqueria...,112,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 40.7612033639422, 'longitude': -7...","[delivery, pickup]",$,"{'address1': '32-44 31st St', 'address2': '', ...",+17187775648,(718) 777-5648,714.301080
7,anBWvFPOce4nl3qLX-H6Rg,el-mero-mero-long-island-city,El Mero Mero,https://s3-media2.fl.yelpcdn.com/bphoto/7tp30k...,False,https://www.yelp.com/biz/el-mero-mero-long-isl...,81,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 40.753834323302, 'longitude': -73...","[delivery, pickup]",$$,"{'address1': '35-57 38th St', 'address2': '', ...",+17183922444,(718) 392-2444,1457.114073
8,jeWIYbgBho9vBDhc5S1xvg,chanos-cantina-astoria,Chano's Cantina,https://s3-media1.fl.yelpcdn.com/bphoto/B34FXj...,False,https://www.yelp.com/biz/chanos-cantina-astori...,163,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.0,"{'latitude': 40.756621, 'longitude': -73.929336}","[delivery, restaurant_reservation, pickup]",$$,"{'address1': '35-55 31st', 'address2': '', 'ad...",+19178327261,(917) 832-7261,1290.427488
9,6rM-N1SBbKPC7DWZNg5awQ,los-portales-astoria-2,Los Portales,https://s3-media1.fl.yelpcdn.com/bphoto/w1OHgt...,False,https://www.yelp.com/biz/los-portales-astoria-...,259,"[{'alias': 'mexican', 'title': 'Mexi

In [5]:
data.price.replace(to_replace={'$': 'price_cheap', 
                               '$$': 'price_medium', 
                               '$$$': 'price_expensive', 
                               '$$$$': 'price_fancy'}, inplace=True)

In [6]:
categories=[]

for i in data.index:
    biz_cats=f'{data.price[i]} '
    
    for cat in data.categories[i]:
        biz_cats += f'{cat["title"]} '
    
    categories.append(biz_cats.strip())

In [7]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(categories)

In [8]:
true_k = 5
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=5, n_init=1, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [9]:
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

In [10]:
rests = {1:[], 2:[], 3:[], 4:[], 5:[]}
cats = {1:[], 2:[], 3:[], 4:[], 5:[]}

i=0
for cat in categories:
    X = vectorizer.transform([cat])
    predicted = model.predict(X)
    rests[predicted[0]+1].append(data['name'][i])
    cats[predicted[0]+1] += cat.split()
    i+=1

price = data['price'][data['name']==company_name].values[0]
X = vectorizer.transform([f'{term} {price}'])
predicted = model.predict(X)

for i in range(5):
    if company_name in rests[i+1] or i==predicted:
        new_cats = list(set(cats[i+1]))
        new_cats.sort()
        print(f'\nGroup {i+1}: {" ".join(new_cats)}')
        rests[i+1].sort()
        for rest in rests[i+1]:
            print(' ' + rest)


Group 2: Chicken Mexican Peruvian Salad Seafood Southern Tacos Tex-Mex Wings price_expensive price_medium
 Calexico - Upper East Side
 Casa Enrique
 Cemitas El Tigre
 Chihuahua
 De Mole
 De Mole Astoria
 El Mero Mero
 El Rey Restaurant
 Hidalgo Mexican Food
 Los Amigos Mexican Restaurant
 Mezquite Restaurant
 Mojave
 Noche De Margaritas
 Oxomoco
 Taqueria Coatzingo
 Toloache 82nd

Group 5: & Bars Breakfast Brunch Cocktail Cuisine Event Lounges Mexican New Spaces Tacos Venues Wine price_medium
 Chano's Cantina
 Chela & Garnacha
 Conmigo
 Corazon De Mexico Restaurant & Bar
 Juquila Kitchen and Bar
 Las Catrinas Mexican Bar & Eatery
 Las Margaritas Astoria Restaurant
 Maizal Restaurant & Tequila Bar
 Mexicue
